# Forward Planner

## Unify

Use the accompanying `unification.py` file for unification. For this assignment, you're almost certainly going to want to be able to:

1. specify the problem in terms of S-expressions.
2. parse them.
3. work with the parsed versions.

`parse` and `unification` work exactly like the programming assignment for last time.

In [1]:
from unification import parse, unification, is_variable

## list_to_string
The `list_to_string` function converts a nested list structure into a formatted string representation. This is commonly used in symbolic logic, planning, and AI systems to create readable action or condition strings from list-based expressions. The function handles nested lists by recursively concatenating elements within parentheses.

## Args
* **exp**: A list or string expression. If it is a list, the function recursively processes each element to construct a nested string. If it is already a string, it is returned directly.

## Returns
* **result**: A string representation of the nested list structure. Elements within lists are separated by spaces, and each sub-list is enclosed in parentheses.


In [2]:
def list_to_string(exp):
    if isinstance(exp, list):
        return "(" + " ".join(list_to_string(e) for e in exp) + ")"
    return exp

In [3]:
# Test for list_to_string
assert list_to_string(['action', 'Me', 'Home']) == "(action Me Home)"
assert list_to_string(['move', ['agent', 'Me'], 'Store']) == "(move (agent Me) Store)"


## apply_substitution
The `apply_substitution` function applies variable substitutions to a given expression. This is useful in symbolic logic or AI systems where variables within expressions need to be replaced with specific values according to a substitution map. The function supports expressions in both string and list formats and applies substitutions recursively for nested lists.

## Args
* **exp**: The expression to which substitutions will be applied. This can be a string or a nested list structure.
* **substitution**: A dictionary containing variable-value pairs. Variables in the expression that match keys in this dictionary will be replaced by their corresponding values.

## Returns
* **unified_exp**: A string representation of the expression after substitutions have been applied. If `exp` is a nested list, the function recursively applies substitutions and formats it as a single string with the `list_to_string` helper function.


In [4]:
def apply_substitution(exp, substitution):
    if is_variable(exp):
        return substitution.get(exp, exp)
    if isinstance(exp, str):
        parsed_exp = parse(exp)
    else:
        parsed_exp = exp
    unified_exp = []
    for term in parsed_exp:
        if is_variable(term):
            unified_term = substitution.get(term, term)
            unified_exp.append(unified_term)
        elif isinstance(term, list):
            unified_exp.append(apply_substitution(term, substitution))
        else:
            unified_exp.append(term)
    return list_to_string(unified_exp)


In [5]:
assert apply_substitution(['?agent', '?place'], {'?agent': 'Me', '?place': 'Home'}) == "(Me Home)"
assert apply_substitution(['action', '?agent', '?place'], {'?agent': 'You', '?place': 'Bank'}) == "(action You Bank)"

## apply_action
The `apply_action` function modifies the current state of a system based on a specified action and a substitution frame. This function is typically used in AI planning algorithms, where actions have preconditions and effects that modify the state. The function removes elements in the state that match any "delete" effects of the action and then adds elements corresponding to the "add" effects.

## Args
* **state**: A list representing the current state of the system. Each element in the list is a string that represents a condition or fact in the current state.
* **action**: A dictionary with "delete" and "add" keys, each containing lists of conditions. The "delete" list specifies which conditions to remove from the state, while the "add" list specifies which conditions to add.
* **frame**: A dictionary that provides substitutions for variables in the action's conditions. The substitutions map variable names to specific values to apply when modifying the state.

## Returns
* **new_state**: A list representing the updated state after applying the action's "delete" and "add" effects, with substitutions applied where necessary.


In [6]:
def apply_action(state, action, frame):
    new_state = []
    for s in state:
        delete_matched = False
        for delete_cond in action["delete"]:
            substituted_delete_cond = apply_substitution(delete_cond, frame)
            if s == substituted_delete_cond:
                delete_matched = True
                break
        if not delete_matched:
            new_state.append(s)
    for add_effect in action["add"]:
        substituted_add_effect = apply_substitution(add_effect, frame)
        new_state.append(substituted_add_effect)

    return new_state

In [7]:
start_state = [
    "(item Saw)",
    "(item Drill)",
    "(place Home)",
    "(place Store)",
    "(place Bank)",
    "(agent Me)",
    "(at Me Home)",
    "(at Saw Store)",
    "(at Drill Store)"
]

The goal state:

In [8]:
goal = [
    "(item Saw)",    
    "(item Drill)",
    "(place Home)",
    "(place Store)",
    "(place Bank)",    
    "(agent Me)",
    "(at Me Home)",
    "(at Drill Me)",
    "(at Saw Store)"    
]

and the actions/operators:

In [9]:
actions = {
    "drive": {
        "action": "(drive ?agent ?from ?to)",
        "conditions": [
            "(agent ?agent)",
            "(place ?from)",
            "(place ?to)",
            "(at ?agent ?from)"
        ],
        "add": [
            "(at ?agent ?to)"
        ],
        "delete": [
            "(at ?agent ?from)"
        ]
    },
    "buy": {
        "action": "(buy ?purchaser ?seller ?item)",
        "conditions": [
            "(item ?item)",
            "(place ?seller)",
            "(agent ?purchaser)",
            "(at ?item ?seller)",
            "(at ?purchaser ?seller)"
        ],
        "add": [
            "(at ?item ?purchaser)"
        ],
        "delete": [
            "(at ?item ?seller)"
        ]
    }
}

## parse_conditions
Parses a list of conditions, converting each condition from a string format into a structured format (e.g., a nested list) using the `parse` function. This allows conditions to be handled and manipulated more easily in symbolic reasoning tasks.

## Args
* **conditions**: List of conditions in string format that need to be parsed into structured format.

## Returns
* **parsed_conditions**: List of parsed conditions, where each condition is converted into a structured format.

In [10]:
def parse_conditions(conditions):
    parsed_conditions = []
    for condition in conditions:
        parsed_conditions.append(parse(condition))
    return parsed_conditions

In [11]:
conditions = ["(at Me Home)", "(has Me Saw)"]
parsed_conditions = parse_conditions(conditions)
assert parsed_conditions == [['at', 'Me', 'Home'], ['has', 'Me', 'Saw']]

## parse_state
Converts each element of a state from a string format into a structured format (e.g., a nested list) using the `parse` function. This parsing allows the state to be processed and matched more flexibly in symbolic reasoning and planning tasks.

## Args
* **state**: List of state elements in string format that need to be parsed into structured format.

## Returns
* **parsed_state**: List of parsed state elements, where each element is converted into a structured format for easier manipulation.

In [12]:
def parse_state(state):
    parsed_state = []
    for item in state:
        parsed_state.append(parse(item))
    return parsed_state

In [13]:
state = ["(at Me Home)", "(has Me Saw)"]
parsed_state = parse_state(state)
assert parsed_state == [['at', 'Me', 'Home'], ['has', 'Me', 'Saw']]

## check_preconditions
Evaluates if a set of conditions can be matched within a given state by attempting to unify each condition with elements of the state. This function uses a substitution frame to track variable matches, updating it progressively as conditions are checked. If any condition cannot be unified with the state, the function returns `False`, signaling that the preconditions are not met. If all conditions are successfully unified, the function returns the updated frame containing the substitutions.

## Args
* **conditions**: List of parsed conditions (structured format) that need to be satisfied in the state.
* **state**: List of parsed state elements in structured format, representing the current state of the world.
* **initial_frame**: Dictionary of initial substitutions (optional), pre-filled with known values to constrain the unification process.
* **debug**: Boolean flag for printing detailed debugging information on the unification process.

## Returns
* **frame**: A dictionary containing the successful substitutions if all conditions are met. Returns `False` if any condition cannot be unified with the state.


In [14]:
def check_preconditions(conditions, state, initial_frame=None, debug=False):
    frame = initial_frame.copy() if initial_frame else {}
    for condition in conditions:
        condition_met = False
        if debug:
            print(f"\nChecking condition: {condition} with initial frame: {frame}")
        for state_element in state:
            result = unification(condition, state_element, frame)
            if debug:
                print(f"Attempting unification between condition {condition} and state element {state_element}\nUnification result: {result}")
            if result is not False:
                frame.update(result)
                condition_met = True
                if debug:
                    print(f"Condition {condition} met with state element {state_element}\nUpdated frame: {frame}")
                break
        if not condition_met:
            if debug:
                print(f"Condition {condition} not met in current state.")
            return False
    if initial_frame:
        for key, value in initial_frame.items():
            if frame.get(key) != value:
                if debug:
                    print(f"Initial frame mismatch for key: {key}, expected: {value}, found: {frame.get(key)}")
                return False
    
    if debug:
        print(f"All conditions met with substitutions: {frame}")
    return frame

In [15]:
state = [
    ["agent", "Me"],
    ["place", "Home"],
    ["place", "Store"],
    ["at", "Me", "Home"]
]
conditions = [
    ["agent", "?agent"],
    ["place", "?from"],
    ["place", "?to"],
    ["at", "?agent", "?from"]
]
result = check_preconditions(conditions, state, debug=True)
print("Preconditions Met:", result)


Checking condition: ['agent', '?agent'] with initial frame: {}
Attempting unification between condition ['agent', '?agent'] and state element ['agent', 'Me']
Unification result: {'?agent': 'Me'}
Condition ['agent', '?agent'] met with state element ['agent', 'Me']
Updated frame: {'?agent': 'Me'}

Checking condition: ['place', '?from'] with initial frame: {'?agent': 'Me'}
Attempting unification between condition ['place', '?from'] and state element ['agent', 'Me']
Unification result: False
Attempting unification between condition ['place', '?from'] and state element ['place', 'Home']
Unification result: {'?agent': 'Me', '?from': 'Home'}
Condition ['place', '?from'] met with state element ['place', 'Home']
Updated frame: {'?agent': 'Me', '?from': 'Home'}

Checking condition: ['place', '?to'] with initial frame: {'?agent': 'Me', '?from': 'Home'}
Attempting unification between condition ['place', '?to'] and state element ['agent', 'Me']
Unification result: False
Attempting unification betw

## instantiate_action
Generates a string representation of an action with variables substituted based on the provided frame. This function takes an action template and applies substitutions from a given frame, resulting in a fully instantiated action.

## Args
* **action**: A dictionary containing the action template with keys such as `"action"` which represents the structure of the action to be instantiated.
* **frame**: A dictionary of substitutions where each variable (as a string) is mapped to its corresponding value. This frame provides the values for the variables in the action.

## Returns
* **action_str**: A string representing the action after all substitutions have been applied, formatted as a single string using `list_to_string`.

In [16]:
def instantiate_action(action, frame):
    action_str = list_to_string(apply_substitution(action["action"], frame))
    return action_str

In [17]:
action = {"action": ["move", "?agent", "?from", "?to"]}
frame = {"?agent": "Me", "?from": "Home", "?to": "Store"}
assert instantiate_action(action, frame) == "(move Me Home Store)"

action = {"action": ["drive", "?driver", "?origin", "?destination"]}
frame = {"?driver": "Alice", "?origin": "Garage", "?destination": "Work"}
assert instantiate_action(action, frame) == "(drive Alice Garage Work)"

action = {"action": ["pickup", "?item", "?location"]}
frame = {"?item": "Package", "?location": "Reception"}
assert instantiate_action(action, frame) == "(pickup Package Reception)"

## check_and_apply_action
Checks whether an action's preconditions can be satisfied in the given state and, if successful, applies the action's effects to generate a new state. This function returns a list of possible actions and their resulting states, taking into account the initial frame provided for substitutions.

## Args
* **action**: A dictionary containing the action's details, including its preconditions, effects, and structure.
* **state**: A list representing the current state, where each element is a fact or statement.
* **temp_frame**: A dictionary containing initial variable substitutions for the action. It maps variables to specific values to guide unification.
* **debug**: A boolean flag that, when set to `True`, outputs debug information about the unification process and state transformations.

## Returns
* **possible_actions**: A list of tuples, each containing:
  - **instantiated_action**: A string representing the action after applying the substitutions.
  - **new_state**: A list representing the state after applying the action's add and delete effects.


In [18]:
def check_and_apply_action(action, state, temp_frame, debug):
    possible_actions = []
    frame = check_preconditions(parse_conditions(action["conditions"]), parse_state(state), temp_frame, debug)
    if frame is not False:
        full_frame = {**temp_frame, **frame}
        instantiated_action = instantiate_action(action, full_frame)
        new_state = apply_action(state, action, full_frame)
        possible_actions.append((instantiated_action, new_state))
        if debug:
            print(f"Instantiated action: {instantiated_action}")
            print(f"New state: {new_state}\n")
    return possible_actions

## apply_action_effects
Applies the effects of an action to a given state, generating a new state. This function handles both "delete" and "add" effects as specified in the action, based on the variable substitutions in the provided frame.

## Args
* **state**: A list representing the current state, where each element is a fact or statement.
* **action**: A dictionary containing the action's effects, specifically lists of statements to "add" and "delete" in the state.
* **frame**: A dictionary of variable substitutions for the action. This is used to substitute variables in the action effects with specific values.

## Returns
* **new_state**: A list representing the modified state after applying the action's add and delete effects. Each effect is applied with variable substitutions as specified in the frame.

In [19]:
def apply_action_effects(state, action, frame):
    new_state = list(state)
    for effect in action["delete"]:
        parsed_effect = parse(effect)
        substituted_effect = list_to_string(apply_substitution(parsed_effect, frame))
        if substituted_effect in new_state:
            new_state.remove(substituted_effect)
    for effect in action["add"]:
        parsed_effect = parse(effect)
        substituted_effect = list_to_string(apply_substitution(parsed_effect, frame))
        if substituted_effect not in new_state:
            new_state.append(substituted_effect)

    return new_state

In [20]:
state = ["(at Me Home)", "(has Me Saw)"]
action = {
    "delete": ["(at Me Home)"],
    "add": ["(at Me Store)"]
}
frame = {"?agent": "Me", "?place": "Home", "?to": "Store"}
assert apply_action_effects(state, action, frame) == ["(has Me Saw)", "(at Me Store)"]

## instantiate_action_with_frame
Generates a string representation of an action after applying variable substitutions from a given frame. This function uses the frame to replace any variables in the action template with their specific values.

## Args
* **action**: A dictionary representing the action, containing an `"action"` key with the action's template as a list structure.
* **frame**: A dictionary containing variable substitutions, where each key is a variable name and each value is the specific substitution.

## Returns
* **action_str**: A string representing the instantiated action with variables replaced by specific values from the frame.

In [21]:
def instantiate_action_with_frame(action, frame):
    return list_to_string(apply_substitution(action["action"], frame))

## get_places
Extracts all unique place names from the parsed state, identifying any elements that are categorized as `"place"`.

## Args
* **parsed_state**: A list of parsed facts, where each fact is a list structure representing an element in the state. Each element is expected to have its type (like `"place"`) as the first item and details like the name of the place as subsequent items.

## Returns
* **places**: A list of unique place names extracted from the parsed state.

In [22]:
def get_places(parsed_state):
    places = []
    for fact in parsed_state:
        if fact[0] == "place":
            places.append(fact[1])
    return places

In [23]:
parsed_state = [['place', 'Home'], ['at', 'Me', 'Home'], ['place', 'Store']]
assert get_places(parsed_state) == ['Home', 'Store']

## generate_buy_actions
Generates all feasible `"buy"` actions based on the provided action structure, state conditions, and available places. This function iterates over each place in the state and attempts to match it with items available for purchase. It checks if each item at a place meets the action's conditions and generates a new instantiated action and resulting state if the conditions are satisfied.

## Args
* **action**: Dictionary containing the action's `"conditions"`, `"delete"`, and `"add"` effects.
* **parsed_conditions**: List of parsed conditions from the action to determine if the action can be executed in the current state.
* **parsed_state**: List of parsed state facts representing the current environment.
* **places**: List of places available in the state, extracted using `get_places`.
* **state**: List of strings representing the current environment's state.
* **debug**: Boolean flag for debugging output. If `True`, prints step-by-step checks of the conditions and actions generated.

## Returns
* **possible_actions**: List of tuples, where each tuple contains:
  - `instantiated_action`: Instantiated action string with applied substitutions.
  - `new_state`: Resulting state list after applying the action's effects.
  - `full_frame`: Dictionary representing the substitution frame used for the action.

For each place in `places`, this function iterates over items in the state to find potential purchases. It creates a temporary frame setting the `?seller` to the current place and the `?item` to each item. If all preconditions for the action are met, it applies substitutions to create the instantiated action and determines the new state by applying action effects.

In [24]:
def generate_buy_actions(action, parsed_conditions, parsed_state, places, state, debug=False):
    possible_actions = []
    for place in places:
        temp_frame = {"?seller": place}
        for state_element in state:
            if state_element.startswith("(item "):
                item_name = state_element.split()[1].strip(")")
                item_frame = temp_frame.copy()
                item_frame["?item"] = item_name
                
                if debug:
                    print(f"Checking place: {place} with item: {item_name}")
                frame = check_preconditions(parsed_conditions, parsed_state, item_frame, debug)
                if frame is not False:
                    full_frame = {**item_frame, **frame}
                    instantiated_action = instantiate_action_with_frame(action, full_frame)
                    new_state = apply_action_effects(state, action, full_frame)
                    possible_actions.append((instantiated_action, new_state, full_frame))
                    
                    if debug:
                        print(f"Generated possible action: {instantiated_action}")
                        print(f"New possible state: {new_state}\n")

    return possible_actions

In [25]:
actions = {
    "buy": {
        "conditions": ["(at ?purchaser ?seller)", "(at ?item ?seller)", "(agent ?purchaser)", "(item ?item)", "(place ?seller)"],
        "action": ["buy", "?purchaser", "?seller", "?item"],
        "delete": ["(at ?item ?seller)"],
        "add": ["(has ?purchaser ?item)"]
    }
}
parsed_conditions = parse_conditions(actions["buy"]["conditions"])
parsed_state = parse_state(["(place Store)", "(agent Me)", "(at Me Store)", "(item Saw)", "(at Saw Store)"])
state = ["(place Store)", "(agent Me)", "(at Me Store)", "(item Saw)", "(at Saw Store)"]
places = get_places(parsed_state)
buy_actions = generate_buy_actions(actions["buy"], parsed_conditions, parsed_state, places, state)
assert len(buy_actions) == 1
assert buy_actions[0][0] == "(buy Me Store Saw)"

## generate_drive_actions
Generates all feasible `"drive"` actions based on the provided action structure, current conditions, and available places. This function iterates over each combination of places, identifying pairs where movement is possible, and verifies if the action's conditions are met. If conditions are satisfied, a new instantiated action is created, and the resulting state after executing the action is generated.

## Args
* **action**: Dictionary containing the action's `"conditions"`, `"delete"`, and `"add"` effects.
* **parsed_conditions**: List of parsed conditions from the action to determine if the action can be executed in the current state.
* **parsed_state**: List of parsed state facts representing the current environment.
* **places**: List of places available in the state, extracted using `get_places`.
* **state**: List of strings representing the current environment's state.
* **debug**: Boolean flag for debugging output. If `True`, prints step-by-step checks of the conditions and actions generated.

## Returns
* **possible_actions**: List of tuples, where each tuple contains:
  - `instantiated_action`: Instantiated action string with applied substitutions.
  - `new_state`: Resulting state list after applying the action's effects.
  - `full_frame`: Dictionary representing the substitution frame used for the action.

For each `from_place` and `to_place` pair where `from_place` is different from `to_place`, this function sets up a temporary frame with `?from` and `?to` placeholders. It verifies if all preconditions for the drive action are met for this pair of places. Upon meeting the conditions, it instantiates the action with substitutions and determines the new state by applying the action's effects.

In [26]:
def generate_drive_actions(action, parsed_conditions, parsed_state, places, state, debug=False):
    possible_actions = []
    for from_place in places:
        for to_place in places:
            if from_place != to_place:
                temp_frame = {"?from": from_place, "?to": to_place}
                frame = check_preconditions(parsed_conditions, parsed_state, temp_frame, debug)
                if frame is not False:
                    full_frame = {**temp_frame, **frame}
                    instantiated_action = instantiate_action_with_frame(action, full_frame)
                    new_state = apply_action_effects(state, action, full_frame)
                    possible_actions.append((instantiated_action, new_state, full_frame))
                    if debug:
                        print(f"Generated possible action: {instantiated_action}")
                        print(f"New possible state: {new_state}\n")
    return possible_actions

In [27]:
actions = {
    "drive": {
        "conditions": ["(at ?agent ?from)", "(place ?from)", "(place ?to)", "(agent ?agent)"],
        "action": ["drive", "?agent", "?from", "?to"],
        "delete": ["(at ?agent ?from)"],
        "add": ["(at ?agent ?to)"]
    }
}
parsed_conditions = parse_conditions(actions["drive"]["conditions"])
parsed_state = parse_state(["(place Home)", "(place Store)", "(agent Me)", "(at Me Home)"])
state = ["(place Home)", "(place Store)", "(agent Me)", "(at Me Home)"]
places = get_places(parsed_state)
drive_actions = generate_drive_actions(actions["drive"], parsed_conditions, parsed_state, places, state,True)
assert len(drive_actions) == 1
assert drive_actions[0][0] == "(drive Me Home Store)"


Checking condition: ['at', '?agent', '?from'] with initial frame: {'?from': 'Home', '?to': 'Store'}
Attempting unification between condition ['at', '?agent', '?from'] and state element ['place', 'Home']
Unification result: False
Attempting unification between condition ['at', '?agent', '?from'] and state element ['place', 'Store']
Unification result: False
Attempting unification between condition ['at', '?agent', '?from'] and state element ['agent', 'Me']
Unification result: False
Attempting unification between condition ['at', '?agent', '?from'] and state element ['at', 'Me', 'Home']
Unification result: {'?from': 'Home', '?to': 'Store', '?agent': 'Me'}
Condition ['at', '?agent', '?from'] met with state element ['at', 'Me', 'Home']
Updated frame: {'?from': 'Home', '?to': 'Store', '?agent': 'Me'}

Checking condition: ['place', '?from'] with initial frame: {'?from': 'Home', '?to': 'Store', '?agent': 'Me'}
Attempting unification between condition ['place', '?from'] and state element ['pl

In [28]:
action = actions["drive"]
print(action)
parsed_conditions = parse_conditions(action["conditions"])
print(parsed_conditions)
parsed_state = parse_state(start_state)
print(parsed_state)
places = get_places(parsed_state)
drive_actions = generate_drive_actions(action, parsed_conditions, parsed_state, places, start_state, debug=True)
for item in drive_actions:
    print(f'{item[0]}\n{item[1]}\n{item[2]}\n')

{'conditions': ['(at ?agent ?from)', '(place ?from)', '(place ?to)', '(agent ?agent)'], 'action': ['drive', '?agent', '?from', '?to'], 'delete': ['(at ?agent ?from)'], 'add': ['(at ?agent ?to)']}
[['at', '?agent', '?from'], ['place', '?from'], ['place', '?to'], ['agent', '?agent']]
[['item', 'Saw'], ['item', 'Drill'], ['place', 'Home'], ['place', 'Store'], ['place', 'Bank'], ['agent', 'Me'], ['at', 'Me', 'Home'], ['at', 'Saw', 'Store'], ['at', 'Drill', 'Store']]

Checking condition: ['at', '?agent', '?from'] with initial frame: {'?from': 'Home', '?to': 'Store'}
Attempting unification between condition ['at', '?agent', '?from'] and state element ['item', 'Saw']
Unification result: False
Attempting unification between condition ['at', '?agent', '?from'] and state element ['item', 'Drill']
Unification result: False
Attempting unification between condition ['at', '?agent', '?from'] and state element ['place', 'Home']
Unification result: False
Attempting unification between condition ['at',

## generate_possible_actions
Generates all possible actions of a specified type (`"drive"` or `"buy"`) based on the current state, action definitions, and preconditions. This function checks the feasibility of the specified action by evaluating its conditions and then instantiates valid actions with appropriate substitutions.

## Args
* **action_name**: The name of the action type to generate (e.g., `"drive"` or `"buy"`).
* **actions**: Dictionary containing action definitions, each with its respective `"conditions"`, `"delete"`, and `"add"` effects.
* **state**: List of strings representing the current environment's state.
* **debug**: Boolean flag for debugging output. If `True`, provides detailed information on actions generated and conditions checked.

## Returns
* **possible_actions**: List of tuples, where each tuple contains:
  - `instantiated_action`: The instantiated action string after substitutions have been applied.
  - `new_state`: The resulting state list after applying the action's effects.
  - `full_frame`: A dictionary representing the substitution frame used for the action.

In [29]:
def generate_possible_actions(action_name, actions, state, debug=False):
    action = actions[action_name]
    parsed_conditions = parse_conditions(action["conditions"])
    parsed_state = parse_state(state)
    places = get_places(parsed_state)

    possible_actions = []

    if action_name == "drive":
        drive_actions = generate_drive_actions(action, parsed_conditions, parsed_state, places, state, debug)
        for instantiated_action, new_state, full_frame in drive_actions:
            possible_actions.append((instantiated_action, new_state, full_frame))
    elif action_name == "buy":
        buy_actions = generate_buy_actions(action, parsed_conditions, parsed_state, places, state, debug)
        for instantiated_action, new_state, full_frame in buy_actions:
            possible_actions.append((instantiated_action, new_state, full_frame))

    return possible_actions

In [30]:
drive_results = generate_possible_actions("drive", actions, start_state, debug=True)
for item in drive_results:
    print(f'\n {item[0]}\n{item[1]}\n{item[2]}\n')


Checking condition: ['at', '?agent', '?from'] with initial frame: {'?from': 'Home', '?to': 'Store'}
Attempting unification between condition ['at', '?agent', '?from'] and state element ['item', 'Saw']
Unification result: False
Attempting unification between condition ['at', '?agent', '?from'] and state element ['item', 'Drill']
Unification result: False
Attempting unification between condition ['at', '?agent', '?from'] and state element ['place', 'Home']
Unification result: False
Attempting unification between condition ['at', '?agent', '?from'] and state element ['place', 'Store']
Unification result: False
Attempting unification between condition ['at', '?agent', '?from'] and state element ['place', 'Bank']
Unification result: False
Attempting unification between condition ['at', '?agent', '?from'] and state element ['agent', 'Me']
Unification result: False
Attempting unification between condition ['at', '?agent', '?from'] and state element ['at', 'Me', 'Home']
Unification result: {'

## check_goal
Compares the current state to a goal state to determine if they are equivalent. This function checks if the current state has achieved all the conditions specified in the goal by sorting and comparing both lists.

## Args
* **state**: List of strings representing the current state of the environment.
* **goal**: List of strings representing the desired goal state.

## Returns
* **bool**: Returns `True` if the sorted lists of `state` and `goal` match exactly, indicating that the current state meets the goal conditions; otherwise, returns `False`.

In [31]:
def check_goal(state, goal):
    return sorted(state) == sorted(goal)

In [32]:
start_state = [
    "(item Saw)",
    "(item Drill)",
    "(place Home)",
    "(place Store)",
    "(place Bank)",
    "(agent Me)",
    "(at Me Home)",
    "(at Saw Store)",
    "(at Drill Store)"
]

goal = [
    "(item Saw)",    
    "(item Drill)",
    "(place Home)",
    "(place Store)",
    "(place Bank)",    
    "(agent Me)",
    "(at Me Home)",
    "(at Drill Me)",
    "(at Saw Store)"    
]

actions = {
    "drive": {
        "action": "(drive ?agent ?from ?to)",
        "conditions": [
            "(agent ?agent)",
            "(place ?from)",
            "(place ?to)",
            "(at ?agent ?from)"
        ],
        "add": [
            "(at ?agent ?to)"
        ],
        "delete": [
            "(at ?agent ?from)"
        ]
    },
    "buy": {
        "action": "(buy ?purchaser ?seller ?item)",
        "conditions": [
            "(item ?item)",
            "(place ?seller)",
            "(agent ?purchaser)",
            "(at ?item ?seller)",
            "(at ?purchaser ?seller)"
        ],
        "add": [
            "(at ?item ?purchaser)"
        ],
        "delete": [
            "(at ?item ?seller)"
        ]
    }
}

## forward_planner
A planner function that attempts to reach a goal state from a starting state using a set of actions. This function employs a depth-first search approach, tracking explored states to avoid revisiting them. At each step, it checks if the current state matches the goal and, if not, generates possible actions from the current state and continues exploring. The function also supports optional debugging output. This function iterates through the frontier of unexplored states, expanding each state by generating possible actions, and checks if each new state meets the goal conditions. It avoids revisiting states by keeping track of explored states using a set.

## Args
* **start_state**: List of strings representing the initial state of the environment.
* **goal**: List of strings representing the desired goal state.
* **actions**: Dictionary of available actions, where each action contains preconditions and effects.
* **debug** (optional): Boolean flag for printing debug information during execution. Defaults to `False`.

## Returns
* **list**: Returns a list of actions (as strings) that form a path from the `start_state` to the `goal` state if found. If the goal is not reachable, returns an empty list.

In [33]:
def forward_planner(start_state, goal, actions, debug=False):
    explored = set()
    frontier = [(start_state, [])]
    while frontier:
        current_state, path = frontier.pop()
        canonical_state = tuple(sorted(current_state))
        if debug:
            print(f"Current state: {current_state}")
            print(f"Path so far: {path}\n")
        if check_goal(current_state, goal):
            return path
        explored.add(canonical_state)
        for action_name in actions:
            if debug: print(f'Action: {action_name}')
            possible_actions = generate_possible_actions(action_name, actions, current_state, debug)
            for instantiated_action, new_state, full_frame in possible_actions:
                canonical_new_state = tuple(sorted(new_state))
                if canonical_new_state not in explored:
                    frontier.append((new_state, path + [instantiated_action]))
                    if debug:
                        print(f"Pushing new state to frontier with action: {instantiated_action}\n")
    return []


In [34]:
plan = forward_planner( start_state, goal, actions, debug=True)
print(plan)

Current state: ['(item Saw)', '(item Drill)', '(place Home)', '(place Store)', '(place Bank)', '(agent Me)', '(at Me Home)', '(at Saw Store)', '(at Drill Store)']
Path so far: []

Action: drive

Checking condition: ['agent', '?agent'] with initial frame: {'?from': 'Home', '?to': 'Store'}
Attempting unification between condition ['agent', '?agent'] and state element ['item', 'Saw']
Unification result: False
Attempting unification between condition ['agent', '?agent'] and state element ['item', 'Drill']
Unification result: False
Attempting unification between condition ['agent', '?agent'] and state element ['place', 'Home']
Unification result: False
Attempting unification between condition ['agent', '?agent'] and state element ['place', 'Store']
Unification result: False
Attempting unification between condition ['agent', '?agent'] and state element ['place', 'Bank']
Unification result: False
Attempting unification between condition ['agent', '?agent'] and state element ['agent', 'Me']
Un

In [35]:
for el in plan:
    print(el)

(drive Me Home Bank)
(drive Me Bank Store)
(buy Me Store Drill)
(drive Me Store Bank)
(drive Me Bank Home)
